In [1]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

builder = (
    SparkSession.builder
        .appName("Kafka Delta Stream")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        # ✅ Include BOTH Delta and Kafka connectors
        .config(
            "spark.jars.packages",
            ",".join([
                "io.delta:delta-core_2.12:3.2.0",
                "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1"
            ])
        )
        .config("spark.sql.shuffle.partitions", "2")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/29 20:02:47 WARN Utils: Your hostname, Abhilashs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.29.62 instead (on interface en0)
25/10/29 20:02:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/homebrew/Cellar/apache-spark/4.0.1_1/libexec/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/abeee/.ivy2.5.2/cache
The jars for the packages stored in: /Users/abeee/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4dbc1a78-4a32-4e65-aeb5-a3dcfe559ea7;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 57ms :: artifacts dl 2ms
	:: modules in us

In [2]:
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "user_stats_info") \
    .load()

df.printSchema()


AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

25/10/29 20:54:58 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 588915 ms exceeds timeout 120000 ms
25/10/29 20:54:58 WARN SparkContext: Killing executors is not supported by current scheduler.
25/10/29 21:10:04 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [13]:
delta_path = "/Users/abeee/data_engineering/data/delta_tables/test_table"
df_table=spark.read.format("delta").load(delta_path)
df_table.show(truncate=False)

+---+-------+---+
|id |name   |age|
+---+-------+---+
|1  |Alice  |35 |
|4  |BobAbhi|31 |
|3  |Cathy  |25 |
|2  |Bob    |31 |
+---+-------+---+



In [12]:
from delta.tables import DeltaTable
df=spark.createDataFrame([(1, "Alice", 35),(4, "BobAbhi", 31)], ["id", "name", "age"])
delta_table=DeltaTable.forPath(spark, delta_path)
delta_table.alias("target").merge(
    df.alias("source"),
    "target.id==source.id",).\
        whenMatchedUpdate(set=
        {
            "name":"source.name",
            "age":"source.age"
        }).\
        whenNotMatchedInsert(values={
            "id":"source.id",
            "name":"source.name",
            "age":"source.age"
        }).execute()

25/10/28 17:06:02 WARN MapPartitionsRDD: RDD 57 was locally checkpointed, its lineage has been truncated and cannot be recomputed after unpersisting


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [9]:
spark.stop()